In [1]:
#Import Dependencies
import requests
import pandas as pd
from character_lists import char_names
import csv
import json

In [2]:
# def CustomParser(data):
#     j1 = json.loads(data)
#     return j1

In [3]:
#Read in csv and character list
j = "comics.csv"
comics = pd.read_csv(j)
comics.drop(comics.columns[0], axis=1, inplace=True)
comics.head()

,ID,Title,Date,Characters,Creators
0,82967,Marvel Previews (2017),"[{'type': 'onsaleDate', 'date': '2099-10-30T00...",[],[{'resourceURI': 'http://gateway.marvel.com/v1...
1,82965,Marvel Previews (2017),"[{'type': 'onsaleDate', 'date': '2099-08-28T00...",[],[]
2,82970,Marvel Previews (2017),"[{'type': 'onsaleDate', 'date': '2099-01-29T00...",[],[{'resourceURI': 'http://gateway.marvel.com/v1...
3,58586,The Amazing Spider-Man (2015) #19 (Veregge Bla...,"[{'type': 'onsaleDate', 'date': '2029-12-31T00...",[],[{'resourceURI': 'http://gateway.marvel.com/v1...
4,1220,Amazing Spider-Man 500 Covers Slipcase - Book ...,"[{'type': 'onsaleDate', 'date': '2029-12-31T00...",[],[]


In [4]:
#Convert strings in df columns to series and delete old columns
#Start with dates
list_of_dates=[]
comic_date = comics['Date']
for indiv_date in comic_date:
    date_list = eval(indiv_date)
    list_of_dates.append(date_list)
date_series = pd.Series(list_of_dates)
comics['Date Series'] = date_series
#Repeat with characters
list_of_chars=[]
comic_char = comics['Characters']
for indiv_char in comic_char:
    char_list = eval(indiv_char)
    list_of_chars.append(char_list)
char_series = pd.Series(list_of_chars)
comics['Character Series'] = char_series
#Repeat with creators
list_of_creates=[]
comic_creates = comics['Creators']
for indiv_create in comic_creates:
    create_list = eval(indiv_create)
    list_of_creates.append(create_list)
create_series = pd.Series(list_of_creates)
comics['Creator Series'] = create_series
comics.drop(comics.columns[[2, 3, 4]], axis=1, inplace=True)
comics.head()

,ID,Title,Date Series,Character Series,Creator Series
0,82967,Marvel Previews (2017),"[{'type': 'onsaleDate', 'date': '2099-10-30T00...",[],[{'resourceURI': 'http://gateway.marvel.com/v1...
1,82965,Marvel Previews (2017),"[{'type': 'onsaleDate', 'date': '2099-08-28T00...",[],[]
2,82970,Marvel Previews (2017),"[{'type': 'onsaleDate', 'date': '2099-01-29T00...",[],[{'resourceURI': 'http://gateway.marvel.com/v1...
3,58586,The Amazing Spider-Man (2015) #19 (Veregge Bla...,"[{'type': 'onsaleDate', 'date': '2029-12-31T00...",[],[{'resourceURI': 'http://gateway.marvel.com/v1...
4,1220,Amazing Spider-Man 500 Covers Slipcase - Book ...,"[{'type': 'onsaleDate', 'date': '2029-12-31T00...",[],[]


In [5]:
#Extract Earliest Date and replace
list_of_dates = []
for i in comics['Date Series']:
    j = i[0]['date']
    k = j[0:4]
    k= int(k)
    if  (k > 1900) & (k < 2030):
        list_of_dates.append(k)
    else:
        list_of_dates.append(0)
date_series = pd.Series(list_of_dates)
comics['Date'] = date_series
print(date_series)
comics.drop(comics.columns[2], axis=1, inplace=True)
comics.tail()

0           0
1           0
2           0
3        2029
4        2029
         ... 
48795       0
48796       0
48797       0
48798       0
48799       0
Length: 48800, dtype: int64


,ID,Title,Character Series,Creator Series,Date
48795,19186,Millie the Model (1945) #50,[],[],0
48796,19187,Millie the Model (1945) #51,[],[],0
48797,19188,Millie the Model (1945) #52,[],[],0
48798,19189,Millie the Model (1945) #53,[],[],0
48799,19190,Millie the Model (1945) #54,[],[],0


In [28]:
#flatten df by making rows for creators (but preserve one row so flattening on characters can occur)
flat_comics_df = pd.DataFrame()
flat_id_list = []
flat_title_list = []
flat_date_list = []
flat_creators_list = []
flat_characters_list = []
for i, v in comics.iterrows():
    # try: #requires id, title, and date to do other operations
   
    try: #try looping through creators, and nested looping through characters
                creators = v[3]
                for j in creators:
                    flat_creator = j['name']
                    try: #adds new row for each character that includes current creator
                        characters = v[2]
                        for k in characters:
                            flat_char = k['name']
                            flat_characters_list.append(flat_char)
                            flat_creators_list.append(flat_creator)
                            flat_id_list.append(v[0])
                            flat_title_list.append(v[1])
                            flat_date_list.append(v[4])
                    except:
                            flat_characters_list.append("")
                            flat_creators_list.append(flat_creator)
                            flat_id_list.append(v[0])
                            flat_title_list.append(v[1])
                            flat_date_list.append(v[4])                   
    except:
                    try:
                        characters = v[2]
                        for k in characters:
                            flat_char = k['name']
                            flat_characters_list.append(flat_char)
                            flat_creators_list.append("")
                            flat_id_list.append(v[0])
                            flat_title_list.append(v[1])
                            flat_date_list.append(v[4])
                    except:
                        pass
dict = {"ID": flat_id_list, "Title": flat_title_list, "Date": flat_date_list, "Creator": flat_creators_list, "Character": flat_characters_list}
cdcc_df = pd.DataFrame(dict)
cdcc_df.to_csv("ccdc_df.csv")


In [53]:
for title in cdcc_df['Title']:
    title_slice = title[-5:-1]
    try:
      int_slice = int(title_slice)
      if  (int_slice > 1900) & (int_slice < 2030):
        cdcc_df.loc[cdcc_df['Title'] == title]['Date'] = int_slice
        print(cdcc_df.loc[cdcc_df['Title'] == title]['Date'])
    except:
        pass


    ..
502152    0
502153    0
502154    0
502155    0
502156    0
Name: Date, Length: 65, dtype: int64
502092    0
502093    0
502094    0
502095    0
502096    0
         ..
502152    0
502153    0
502154    0
502155    0
502156    0
Name: Date, Length: 65, dtype: int64
502092    0
502093    0
502094    0
502095    0
502096    0
         ..
502152    0
502153    0
502154    0
502155    0
502156    0
Name: Date, Length: 65, dtype: int64
502092    0
502093    0
502094    0
502095    0
502096    0
         ..
502152    0
502153    0
502154    0
502155    0
502156    0
Name: Date, Length: 65, dtype: int64
502092    0
502093    0
502094    0
502095    0
502096    0
         ..
502152    0
502153    0
502154    0
502155    0
502156    0
Name: Date, Length: 65, dtype: int64
502092    0
502093    0
502094    0
502095    0
502096    0
         ..
502152    0
502153    0
502154    0
502155    0
502156    0
Name: Date, Length: 65, dtype: int64
502092    0
502093    0
502094    0
502095    0
502

In [54]:
cdcc_df.head()

,ID,Title,Date,Creator,Character
0,17486,X-Men: Phoenix - Warsong (2006),2029,Vc Joe Caramagna,Beast
1,17486,X-Men: Phoenix - Warsong (2006),2029,Vc Joe Caramagna,Colossus
2,17486,X-Men: Phoenix - Warsong (2006),2029,Vc Joe Caramagna,Cyclops
3,17486,X-Men: Phoenix - Warsong (2006),2029,Vc Joe Caramagna,Emma Frost
4,17486,X-Men: Phoenix - Warsong (2006),2029,Vc Joe Caramagna,Kitty Pryde


In [101]:
list_of_chars=[]
comic_char = comics['Characters']
for indiv_char in comic_char:
    char_list = eval(indiv_char)
    list_of_chars.append(char_list)
type(list_of_chars)

list

In [55]:
#Question: what's the distribution of appearances of Marvel characters?
char_counts = {}
for i in list_of_chars:
    for j in i:
        char_key = j['name']
        if char_key in char_counts.keys():
            char_counts[char_key] = char_counts[char_key]+1
        else:
            char_counts[char_key] = 1
print(char_counts)

, 'Power Pack': 132, 'Mary Jane Watson': 406, 'Silver Sable': 72, 'Ben Urich': 54, 'Alex Power': 19, 'Energizer': 11, 'Lightspeed': 37, 'Marauders': 17, 'Prism': 7, 'Riptide': 12, 'Scalphunter': 18, 'Vertigo (Savage Land Mutate)': 29, 'Cyclops (Ultimate)': 69, 'Human Torch (Ultimate)': 57, 'Invisible Woman (Ultimate)': 54, 'Nightcrawler (Ultimate)': 40, 'Shadowcat (Ultimate)': 10, 'Thing (Ultimate)': 49, 'Wolverine (Ultimate)': 97, 'X-Men (Ultimate)': 187, 'Black Bolt': 162, 'Echo': 33, 'Edwin Jarvis': 107, 'Gorgon': 64, 'Sentry (Robert Reynolds)': 113, 'Silver Samurai': 39, 'Spider-Woman (Jessica Drew)': 271, 'Black Knight (Sir Percy of Scandia)': 107, 'Blade': 96, 'Captain Marvel (Mar-Vell)': 199, 'Dormammu': 53, 'Mad Thinker': 33, 'Sub-Mariner': 544, 'Wong': 88, 'Bishop': 295, 'Rachel Grey': 88, 'Sentinels': 55, 'Bullseye': 130, 'Runaways': 133, 'Dead Girl': 10, 'Gwen Stacy': 78, 'Union Jack (Joseph Chapman)': 24, 'X-Statix': 42, 'John Jameson': 11, 'Colleen Wing': 82, 'Heroes For H

In [61]:
cdcc_df.head()

,ID,Title,Date,Creator,Character
0,17486,X-Men: Phoenix - Warsong (2006),2029,Vc Joe Caramagna,Beast
1,17486,X-Men: Phoenix - Warsong (2006),2029,Vc Joe Caramagna,Colossus
2,17486,X-Men: Phoenix - Warsong (2006),2029,Vc Joe Caramagna,Cyclops
3,17486,X-Men: Phoenix - Warsong (2006),2029,Vc Joe Caramagna,Emma Frost
4,17486,X-Men: Phoenix - Warsong (2006),2029,Vc Joe Caramagna,Kitty Pryde


In [64]:
#See the numbers of appearances of Marvel Characters
char_counts = cdcc_df.groupby('Character')['ID'].count()
print(char_counts)
# import itertools
# char_sort = dict(sorted(char_counts.items(), key=lambda item: item[1], reverse = True))
# N = 500
# chars_500 = dict(itertools.islice(char_sort.items(), N))
# print(chars_500)

Character
3-D Man                        79
A-Bomb (HAS)                   16
A.I.M.                        247
Aaron Stack                    89
Abomination (Emil Blonsky)    360
                             ... 
Zemo                           42
Zodiak                         10
Zombie (Simon Garth)           18
Zuras                          11
Zzzax                          27
Name: ID, Length: 1189, dtype: int64


In [73]:
#See number of unique creators for characters
char_creators = cdcc_df.groupby('Character')['Creator'].nunique()
char_creators.sort_values(ascending=False, inplace=True)
print(char_creators)

Character
Spider-Man                 1819
X-Men                      1618
Wolverine                  1579
Captain America            1504
Iron Man                   1487
                           ... 
Morg                          2
Gargoyle (Yuri Topolov)       2
Hedge Knight                  2
Monster Badoon                2
Kabuki                        1
Name: Creator, Length: 1189, dtype: int64


In [75]:
#See the number of unique characters for creators
creator_chars = cdcc_df.groupby('Creator')['Character'].nunique()
creator_chars.sort_values(ascending=False, inplace = True)
print(creator_chars)

Creator
Richard Starkings               596
Vc Joe Caramagna                587
Jeff Youngquist                 547
Virtual Calligr                 524
Tom Brevoort                    434
                               ... 
Bob Mackie                        1
Robert Sodaro                     1
David Riske                       1
David Navarrot                    1
including Klaus Janson pages      1
Name: Character, Length: 3827, dtype: int64


In [100]:
#DataFrame of Top 500 Marvel Chars with Characters as Index
char_500_df = pd.DataFrame.from_dict(chars_500, orient='index')
char_500_df.head(1400)

,0
Spider-Man,3975
X-Men,3411
Iron Man,2583
Wolverine,2402
Captain America,2384
...,...
Mandroid,1
Zodiak,1
Mordo,1
Lava-Man,1


In [57]:
list_of_creates=[]
comic_creates = comics['Creators']
for indiv_create in comic_creates:
    create_list = eval(indiv_create)
    list_of_creates.append(create_list)
print(list_of_creates)

KeyError: 'Creators'

In [80]:
create_counts = {}
for i in list_of_creates:
    for j in i:
        create_key = j['name']
        if create_key in create_counts.keys():
            create_counts[create_key] = create_counts[create_key]+1
        else:
            create_counts[create_key] = 1
print(create_counts)

CARL POTTS JIM LEE': 1, 'Oscar Wilde': 6, 'Paco Roca': 5, 'Avalons Rob Ro': 1, 'Yuko Fukami': 2, 'Mick Gary': 3, 'Mick Gray': 7, 'Jean Simek': 92, 'Blank': 1, 'Charles Nicholas': 16, 'Greg Theakston': 2, 'Top Cow Michael Choi': 2, 'Top Cow Sonia Oback': 2, 'Jonathan Lethem': 9, 'Don Warfield': 144, 'Brian Overton': 1, 'Jim Clark': 4, 'Doug Alexander': 4, 'Mark Parsons': 1, 'HERMAN MELVILLE': 3, 'Ken Barr': 4, 'Seth Kruchkow': 1, 'Bob Burden': 1, 'Kelly Goodine': 1, 'JAMIE TOST': 2, 'Eduardo Risso': 3, 'Karl Rusnak': 6, 'KYLE SIMS': 1, 'Sal Regla': 5, 'Suzanne Gaffney': 61, 'Bruno Fernandez': 5, 'ROBERT RUFFOLO': 1, 'Chris Kipiniak': 1, 'GARY PANTER': 1, 'Mimi Gold': 2, 'Rick Mays': 8, 'J. D. Mettler': 7, 'James Fenimore Cooper': 4, 'Raymond E. Feist': 13, 'Alexandre Dumas': 5, 'Robert Louis Stevenson': 4, 'Nova Studio': 2, 'Roland Bernard Brown': 13, 'Miguel Montenegro': 12, 'JONATHAN LETHEM KARL RUSNAK': 1, 'Jim Reddington': 4, 'Lisa Patrick': 65, 'Julia Bacellar': 1, 'Adriano Batista

In [91]:
#Find top 500 Creators
creator_sort = dict(sorted(create_counts.items(), key=lambda item: item[1], reverse = True))
N = 500
creators_500 = dict(itertools.islice(creator_sort.items(), N))
print(creators_500)


{'Vc Joe Caramagna': 3309, 'Jeff Youngquist': 2977, 'Tom Brevoort': 1876, 'Richard Starkings': 1812, 'Brian Michael Bendis': 1660, 'Stan Lee': 1657, 'Vc Clayton Cowles': 1574, 'Virtual Calligr': 1541, 'Cory Petit': 1235, 'Vc Cory Petit': 1212, 'Nick Lowe': 1205, 'Jordan White': 1186, 'Chris Claremont': 1131, 'Glynis Oliver': 1118, 'Joe Rosen': 1108, 'Peter David': 1048, 'Dave Sharpe': 1039, 'Jack Kirby': 983, 'Bob Sharen': 967, 'Mark Paniccia': 959, 'Roy Thomas': 951, 'Jason Aaron': 859, 'Tom Defalco': 856, 'Sal Buscema': 842, 'Vc Joe Sabino': 840, 'Vc Travis Lanham': 833, 'Mark Bagley': 773, 'Art Simek': 760, 'Al Milgrom': 756, 'Dan Slott': 753, 'Chris Eliopoulos': 745, 'John Romita Jr.': 742, 'Jonathan Hickman': 731, "Frank D'ARMATA": 723, 'John Costanza': 718, 'Mark Waid': 664, 'Wilson Moss': 662, 'Tom Orzechowski': 656, 'Emily Newcomen': 649, 'Salvador Larroca': 633, 'John Byrne': 623, 'Tom Palmer': 616, 'George Roussos': 598, 'Ralph Macchio': 595, 'Greg Pak': 586, 'Jim Novak': 586

In [92]:
#DataFrame of Top 500 Marvel Creators with Creators as Index
creators_500_df = pd.DataFrame.from_dict(creators_500, orient='index')
creators_500_df.head(40)

,0
Vc Joe Caramagna,3309
Jeff Youngquist,2977
Tom Brevoort,1876
Richard Starkings,1812
Brian Michael Bendis,1660
Stan Lee,1657
Vc Clayton Cowles,1574
Virtual Calligr,1541
Cory Petit,1235
Vc Cory Petit,1212


In [98]:
#What characters are the top 500 Creators associated with
cc_df = pd.DataFrame()
for creator in creators_500:
    creator_df = comics.loc[comics["Creators"] ]
creator_df.head()
# for row in comics.iterrows():
#     row_creator = row[5]
#     row_character = row[4]
#     for creators in row_creator:
#         for creator, count in creators_500:
#             if creator==creators:
#                 for character in row_character:
#                     cc_df[creator][character]= cc_df[creator][character]+1

# cc_df.head()


KeyError: 'Vc Joe Caramagna'

In [ ]:
#Question: are Marvel characters old (i.e., were they created long ago)?

In [ ]:
#Question: did Marvel CU characters tend to appear together?

In [ ]:
#Question: what's the distribution over time of appearances of Marvel characters?

In [ ]:
#What distribution of creators is each character associated with?

In [ ]:
#Which creators were most associated with the MCU

In [ ]:
#When did the creators most associated with the MCU create?

In [ ]:
#What does this suggest about the MCU?

In [ ]:
#Before or after Avengers? [categorical]

In [ ]:
#Prominence of before or after Avengers